In [2]:
import MySQLdb
conn = MySQLdb.connect(
    user = 'crawl_usr',
    passwd = 'test001',
    host = 'localhost',
    db = 'crawl_data'
)
print(type(conn))

cursor = conn.cursor()
print(type(cursor))

cursor.execute("CREATE TABLE books (title text, url text)")
conn.commit()

<class 'MySQLdb.connections.Connection'>
<class 'MySQLdb.cursors.Cursor'>


In [3]:
bookname = '처음 시작하는 파이썬'
url = 'www.wikibook.co.kr'
cursor.execute(f'INSERT INTO books VALUES(\"{bookname}\", \"{url}\")')
conn.commit()

#### 실행할때마다 다른 값이 나오지 않게 테이블을 제거하고 다시 생성하는 코드

In [4]:
import MySQLdb
conn = MySQLdb.connect(
    user = 'crawl_usr',
    passwd = 'test001',
    host = 'localhost',
    db = 'crawl_data'
)
# 커서 생성하기
cursor = conn.cursor()

# 실행할 때마다 다른 값이 나오지 않게 테이블을 제거해두기
cursor.execute('DROP TABLE IF EXISTS books')
# 테이블을 생성하기
cursor.execute("CREATE TABLE books (title text, url text)")

# 데이터 저장하기
bookname = '처음 시작하는 파이썬'
url = 'www.wikibook.co.kr'
cursor.execute(f'INSERT INTO books VALUES(\"{bookname}\", \"{url}\")')

# 커밋하기
conn.commit()

# 연결 종료 
conn.close()

### 크롤링 DB에 넣기

In [5]:
import MySQLdb
import requests
from bs4 import BeautifulSoup

In [13]:
if __name__ == '__main__':
    RANK = 100 
    header = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
    req = requests.get('http://www.melon.com/chart/week/index.htm', headers = header)
    html = req.text
    parse = BeautifulSoup(html, 'html.parser')
    
    titles = parse.find_all('div', {'class' : 'ellipsis rank01'})
    singers = parse.find_all('div', {'class' : 'ellipsis rank02'})
    
    title = []
    singer = []
    
    for t in titles:
        title.append(t.find('a').text)
    
    for s in singers:
        singer.append(s.find('span', {'class' : 'checkEllipsis'}).text)
    items = [item for item in zip(title, singer)]

conn = MySQLdb.connect(
    user = 'crawl_usr',
    passwd = 'test001',
    host = 'localhost',
    db = 'crawl_data')

cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS melon')
cursor.execute("CREATE TABLE melon (`rank` int, title text, singer text)")
i = 1

for item in items:
    cursor.execute(
        f'INSERT INTO melon VALUES({i},\"{item[0]}\", \"{item[1]}\")')
    i += 1

conn.commit()
conn.close()